![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/semantic-types.png)

# Semantic Types


Some string values can be recognized as semantic types. For example, email addresses, US zip codes or IP addresses have specific formats that can be recognized, and then split in specific ways.

When getting a DataProfile you can optionally ask to collect counts of values recognized as semantic types. [`Dataflow.get_profile()`](./data-profile.ipynb) executes the Dataflow, calculates profile information, and returns a newly constructed DataProfile. Semantic type counts can be included in the data profile by calling `get_profile` with the `include_stype_counts` argument set to true.

The `stype_counts` property of the DataProfile will then include entries for columns where some semantic types were recognized for some values.

In [ ]:
import azureml.dataprep as dprep
dflow = dprep.read_json(path='../data/json.json')

profile = dflow.get_profile(include_stype_counts=True)

print("row count: " + str(profile.row_count))
profile.stype_counts

To see all the supported semantic types, you can examine the `SType` enumeration. More types will be added over time.

In [ ]:
[t.name for t in dprep.SType]


You can filter the found semantic types down to just those where all non-empty values matched. The `DataProfile.stype_counts` gives a list of semantic type counts for each column, where at least some matches were found. Those lists are in desecending order of count, so here we consider only the first in each list, as that will be the one with the highest count of values that match.

In this example, the column `inspections.business.postal_code` looks to be a US zip code.

In [ ]:
stypes_counts = profile.stype_counts
all_match = [
    (column, stypes_counts[column][0].stype)
    for column in stypes_counts
    if profile.row_count - profile.columns[column].empty_count == stypes_counts[column][0].count
]
all_match

You can use semantic types to compute new columns. The new columns are the values split up into elements, or canonicalized.

Here we reduce our data down to just the `postal` column so we can better see what a `split_stype` operation can do.

In [ ]:
dflow_postal = dflow.keep_columns(['inspections.business.postal_code']).rename_columns({'inspections.business.postal_code': 'postal'})
dflow_postal.head(5)

With `SType.ZipCode`, values are split into their basic five digit zip code and the plus-four add-on of the Zip+4 format.

In [ ]:
dflow_split = dflow_postal.split_stype('postal', dprep.SType.ZIPCODE)
dflow_split.head(5)

`split_stype` also allows you to specify the fields of the stype to use and the name of the new columns. For example, if you just needed to strip the plus four from our zip codes, you could use this.

In [ ]:
dflow_no_plus4 = dflow_postal.split_stype('postal', dprep.SType.ZIPCODE, ['zip'], ['zipNoPlus4'])
dflow_no_plus4.head(5)